In [18]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import mediapipe as mp
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, Dropout
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/validation'

In [4]:

# Function to create dataframe of image paths and labels
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [5]:

# Create dataframes for train and test datasets
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

print(train)
print(test)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]
                                    image     label
0       images/validation\angry\10052.jpg     angry
1       images/validation\angry\10065.jpg     angry
2       im

In [6]:

# Initialize Mediapipe Face Mesh for facial landmarks detection
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

In [7]:

# Function to extract facial landmarks from an image
def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)

    if not results.multi_face_landmarks:
        return np.zeros(468 * 2)  # Return a zero vector if no face is detected

    landmarks = results.multi_face_landmarks[0].landmark
    landmark_array = []
    for landmark in landmarks:
        landmark_array.append(landmark.x)
        landmark_array.append(landmark.y)
    
    return np.array(landmark_array)


In [8]:

# Function to extract combined image features and landmarks
def extract_features_with_landmarks(images):
    features = []
    for image_path in tqdm(images):
        # Load the grayscale image
        img = load_img(image_path, color_mode='grayscale', target_size=(48, 48))
        img = np.array(img)
        img = img.reshape(48, 48, 1)

        # Extract facial landmarks
        landmarks = extract_landmarks(image_path)

        # Flatten the image and concatenate with landmarks
        combined_features = np.concatenate((img.flatten(), landmarks))
        features.append(combined_features)

    features = np.array(features)
    return features

# Extract combined features for training and testing sets
train_features = extract_features_with_landmarks(train['image'])
test_features = extract_features_with_landmarks(test['image'])
# Normalize image pixel values to [0, 1]
x_train = train_features / 255.0
x_test = test_features / 255.0


  0%|          | 0/28821 [00:00<?, ?it/s]w:\Important Documents\Manan\2 UK\MSc\Dissertation\archive\env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
100%|██████████| 7066/7066 [01:22<00:00, 85.63it/s] 


In [9]:

# Encode labels
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

# Convert labels to categorical (one-hot encoding)
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

# Define the model
input_shape = (48 * 48 + 468 * 2,)  # 48x48 grayscale image + 468 landmarks (x, y coordinates)

In [10]:

model = Sequential()
# Fully connected layers (since we flattened the image)
model.add(Dense(1024, activation='relu', input_shape=input_shape))
model.add(Dropout(0.4))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(7, activation='softmax'))


w:\Important Documents\Manan\2 UK\MSc\Dissertation\archive\env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:

# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))


Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.2123 - loss: 2.0003 - val_accuracy: 0.2583 - val_loss: 1.7924
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.2455 - loss: 1.8131 - val_accuracy: 0.2628 - val_loss: 1.7708
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.2483 - loss: 1.7924 - val_accuracy: 0.2764 - val_loss: 1.7489
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.2502 - loss: 1.7820 - val_accuracy: 0.2856 - val_loss: 1.7655
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.2479 - loss: 1.7798 - val_accuracy: 0.2963 - val_loss: 1.7360
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.2457 - loss: 1.7820 - val_accuracy: 0.2871 - val_loss: 1.7338
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.2578 - loss: 1.7777 - val_accuracy: 0.2809 - val_loss: 1.7302
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.2592 - loss: 1.7657 - 

In [13]:

# Save the model architecture and weights
model_json = model.to_json()
with open("emotiondetector_with_landmarks.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector_with_landmarks.h5")


In [14]:

# Function to predict emotion with landmarks
def predict_emotion_with_landmarks(image_path):
    # Extract both grayscale image and landmarks
    img = load_img(image_path, color_mode='grayscale', target_size=(48, 48))
    img = np.array(img).reshape(1, 48 * 48)

    landmarks = extract_landmarks(image_path).reshape(1, 468 * 2)

    # Combine image and landmark features
    combined_features = np.concatenate((img, landmarks), axis=1)
    combined_features = combined_features / 255.0

    # Predict emotion
    pred = model.predict(combined_features)
    return label[pred.argmax()]


In [15]:

# Label mapping for the predictions
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Example prediction
image_path = 'images/train/sad/42.jpg'
predicted_emotion = predict_emotion_with_landmarks(image_path)
print("Model prediction with landmarks is:", predicted_emotion)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step

w:\Important Documents\Manan\2 UK\MSc\Dissertation\archive\env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Model prediction with landmarks is: neutral
